In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/notebook810eb1472d/_output_.zip
/kaggle/input/notebook810eb1472d/__results__.html
/kaggle/input/notebook810eb1472d/__notebook__.ipynb
/kaggle/input/notebook810eb1472d/__output__.json
/kaggle/input/notebook810eb1472d/custom.css
/kaggle/input/notebook810eb1472d/__results___files/__results___2_0.jpg
/kaggle/input/notebook810eb1472d/__results___files/__results___2_0.png


In [2]:
import zipfile
import os

# Path to the uploaded zip file
zip_path = "/kaggle/input/notebook810eb1472d/_output_.zip"
extract_path = "/kaggle/working/frames"

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check contents
os.listdir(extract_path)


['combined_dataset.csv', '__pycache__', 'frames']

In [ ]:
import os
import time
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tqdm import tqdm
import matplotlib.pyplot as plt

# -------------------------
# Load CSV & Clean
# -------------------------
csv_path = "/kaggle/working/frames/combined_dataset.csv"
df = pd.read_csv(csv_path)

df['img_path'] = df['img_path'].apply(lambda x: os.path.join("/kaggle/working/frames/frames", os.path.basename(x)))
df['Label'] = df['Label'].map({0: 0, 5: 1, 10: 2})

tabular_cols = ['EAR', 'MAR', 'Head_Tilt', 'Head_Nod']
df[tabular_cols] = df[tabular_cols].replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=tabular_cols).reset_index(drop=True)

scaler = StandardScaler()
df[tabular_cols] = scaler.fit_transform(df[tabular_cols])

# -------------------------
# Split Dataset
# -------------------------
train_val_df, test_df = train_test_split(df, test_size=0.15, stratify=df['Label'], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df['Label'], random_state=42)

# -------------------------
# Dataset Class
# -------------------------
class DrowsinessCombinedDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['img_path']).convert('RGB')
        if self.transform:
            image = self.transform(image)
        tabular = torch.tensor(row[tabular_cols].astype(np.float32).values, dtype=torch.float32)
        label = torch.tensor(int(row['Label'])).long()
        return image, tabular, label

# -------------------------
# Transforms and DataLoaders
# -------------------------
transform = transforms.Compose([
    transforms.Resize((240, 240)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomApply([transforms.GaussianBlur(5)], p=0.2),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_loader = DataLoader(DrowsinessCombinedDataset(train_df, transform), batch_size=32, shuffle=True, num_workers=2, drop_last=True)

val_loader = DataLoader(DrowsinessCombinedDataset(val_df, transform), batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(DrowsinessCombinedDataset(test_df, transform), batch_size=32, shuffle=False, num_workers=2)

# -------------------------
# Model Class with EfficientNetB1 and BiLSTM
# -------------------------
# -------------------------
# Model Class with EfficientNetB1 and BiLSTM
# -------------------------
class EfficientNetWithTabularBiLSTM(nn.Module):
    def __init__(self, num_tabular_features, num_classes=3):
        super(EfficientNetWithTabularBiLSTM, self).__init__()
        self.efficientnet = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.DEFAULT)
        self.efficientnet = nn.Sequential(*list(self.efficientnet.children())[:-1])
        self.efficientnet_fc_input_dim = 1280
        self.lstm_input_size = self.efficientnet_fc_input_dim + num_tabular_features
        self.bilstm = nn.LSTM(input_size=self.lstm_input_size, hidden_size=128, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128), 
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, num_classes)
        )

    def forward(self, image, tabular):
        x_img = self.efficientnet(image)
        x_img = torch.flatten(x_img, 1)
        x_combined = torch.cat([x_img, tabular], dim=1).unsqueeze(1)
        x_lstm, _ = self.bilstm(x_combined)
        x_lstm = x_lstm[:, -1, :]
        
        # Batch normalization issue when batch size is 1
        if x_lstm.size(0) == 1:
            x_lstm = self.fc[0](x_lstm)  # Linear
            x_lstm = x_lstm.unsqueeze(0) if x_lstm.dim() == 1 else x_lstm
            if x_lstm.size(0) > 1:
                x_lstm = self.fc[1](x_lstm)  # BatchNorm1d
            x_lstm = self.fc[2](x_lstm)  # ReLU
            x_lstm = self.fc[3](x_lstm)  # Dropout
            x_lstm = self.fc[4](x_lstm)  # Final Linear
            return x_lstm.squeeze(0)

        # General case for batch size > 1
        x_lstm = self.fc(x_lstm)
        return x_lstm




# -------------------------
# Training and Evaluation
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNetWithTabularBiLSTM(num_tabular_features=4).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
early_stopping_patience = 5

num_epochs = 10
best_val_acc = 0
patience_counter = 0

train_accuracies = []
val_accuracies = []

best_val_acc = 0  # Track the best validation accuracy

for epoch in range(num_epochs):
    model.train()
    correct, total, total_loss = 0, 0, 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for images, tabular, labels in pbar:
        images, tabular, labels = images.to(device), tabular.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images, tabular)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        batch_acc = 100. * correct / total
        pbar.set_postfix(loss=total_loss/(total+1), accuracy=batch_acc)

    train_acc = 100. * correct / total
    train_accuracies.append(train_acc)

    # Validation phase
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, tabular, labels in val_loader:
            images, tabular, labels = images.to(device), tabular.to(device), labels.to(device)
            outputs = model(images, tabular)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    val_acc = 100. * correct / total
    val_accuracies.append(val_acc)
    scheduler.step(total_loss)

    # Check if the current validation accuracy is the best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print(f"New best model saved with accuracy: {best_val_acc:.2f}%")

    print(f"Epoch {epoch+1}/{num_epochs} - Train Accuracy: {train_acc:.2f}% - Validation Accuracy: {val_acc:.2f}%")

print("Training Complete. Best Validation Accuracy:", max(val_accuracies))

plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.show()


Epoch 1/5: 100%|██████████| 16247/16247 [1:00:10<00:00,  4.50it/s, accuracy=62.7, loss=0.026]


New best model saved with accuracy: 74.76%
Epoch 1/5 - Train Accuracy: 62.69% - Validation Accuracy: 74.76%


Epoch 2/5: 100%|██████████| 16247/16247 [1:00:34<00:00,  4.47it/s, accuracy=74.9, loss=0.0217]


New best model saved with accuracy: 78.58%
Epoch 2/5 - Train Accuracy: 74.93% - Validation Accuracy: 78.58%


Epoch 3/5: 100%|██████████| 16247/16247 [1:00:40<00:00,  4.46it/s, accuracy=78, loss=0.0204] 


New best model saved with accuracy: 81.03%
Epoch 3/5 - Train Accuracy: 78.03% - Validation Accuracy: 81.03%


Epoch 4/5: 100%|██████████| 16247/16247 [57:40<00:00,  4.70it/s, accuracy=79.8, loss=0.0197] 


New best model saved with accuracy: 82.11%
Epoch 4/5 - Train Accuracy: 79.82% - Validation Accuracy: 82.11%


Epoch 5/5: 100%|██████████| 16247/16247 [1:02:31<00:00,  4.33it/s, accuracy=81.2, loss=0.0191]
